# Reto 3

## Miguel Ortiz y Andrea Castiella


In [17]:
import numpy  as np
import pandas as pd
import OurUtils as our
from sklearn.preprocessing import MinMaxScaler

In [18]:
# Read data
X_full = pd.read_csv('Reto 3-Dataset/reto3_trainX.csv', header=0, low_memory=False)
Y_full = pd.read_csv('Reto 3-Dataset/reto3_trainY.csv', header=0, low_memory=False)

print(X_full.head())
print(Y_full.head())

   altitud  azimut  inclinacion  DH_agua  DV_agua  DH_camino  sombra_9am  \
0     3351     354           19      450       95       1064         185   
1     2995      84            7      481       30       5154         231   
2     2884      72           25      210      -45        466         239   
3     3139      76           11      301       58       1368         234   
4     3018     312           17       30       10       1719         172   

   sombra_12pm  sombra_3pm  DH_fuego  ...  t31  t32  t33  t34  t35  t36  t37  \
0          203         153       711  ...    0    0    0    0    0    0    0   
1          227         129      5488  ...    0    0    0    0    0    0    0   
2          183          60      2123  ...    0    0    0    0    0    0    0   
3          220         117      3282  ...    0    0    1    0    0    0    0   
4          225         193      1961  ...    0    0    0    0    0    0    0   

   t38  t39  t40  
0    0    1    0  
1    0    0    0  
2    

## Split dataset
Train split: 80%

Validation split: 20%

In [19]:
# Split into train and validation
valid_size = 0.2
seed = 1234
X_train, Y_train, X_valid, Y_valid = \
   our.single_stratified_split(X_full, Y_full, test_size=valid_size, random_state=seed)

from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler

over = SMOTE(random_state=seed, sampling_strategy={3:10000 , 4:10000 , 5: 10000, 6:10000 , 7:10000})
X_train, Y_train = over.fit_resample(X_train, Y_train)

under = RandomUnderSampler(random_state=seed, sampling_strategy={1:10000 , 2:10000})
X_train, Y_train = over.fit_resample(X_train, Y_train)


## Scale and transform features

In [20]:
from sklearn.preprocessing import PolynomialFeatures

scaler = MinMaxScaler()
X = scaler.fit_transform(X_train)
poly = PolynomialFeatures(2)
X = poly.fit_transform(X)
Y = Y_train.values.ravel()
#X, pca = our.our_PCA(X)


## PCA

In [34]:
# X_proy, pca = our.our_PCA(X)
# X_proy = our.our_kernelPCA(X)
# from imblearn.over_sampling import SMOTE
# sm = SMOTE(random_state=seed, sampling_strategy="minority")
# X_proy, Y = sm.fit_resample(X_proy, Y)

# num_of_pc = len(pca.singular_values_)
# strTitle = '\n First %d principal components' %(num_of_pc)
# print(strTitle)
# print(pca.components_.T)
# strTitle = '\n First %d singular values ' %(num_of_pc)
# print(strTitle)
# print(pca.singular_values_.T)
# strTitle = '\n Explained Variance Ratio'
# print(strTitle)
# print(pca.explained_variance_ratio_.T)
# strTitle = '\n Accumulated Explained Variance Ratio'
# print(strTitle)
# print(np.cumsum(pca.explained_variance_ratio_.T))
# X = X_proy

"\nnum_of_pc = len(pca.singular_values_)\nstrTitle = '\n First %d principal components' %(num_of_pc)\nprint(strTitle)\nprint(pca.components_.T)\nstrTitle = '\n First %d singular values ' %(num_of_pc)\nprint(strTitle)\nprint(pca.singular_values_.T)\nstrTitle = '\n Explained Variance Ratio'\nprint(strTitle)\nprint(pca.explained_variance_ratio_.T)\nstrTitle = '\n Accumulated Explained Variance Ratio'\nprint(strTitle)\nprint(np.cumsum(pca.explained_variance_ratio_.T))"

## Multiclass. OvO vs OvR

In [21]:
# Multiclass
from sklearn.multiclass import OneVsOneClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier

max_depth = 15
base_clf = DecisionTreeClassifier(max_depth = max_depth, random_state=seed)

# base_clf = SVC(kernel='rbf', degree=2, gamma=1, random_state = seed)

# Train normal
clf = base_clf.fit(X,Y)

# Train OvO 
OvO_clf = OneVsOneClassifier(base_clf)
OvO_clf.fit(X,Y)

# Train OvR 
OvR_clf = OneVsRestClassifier(base_clf)
OvR_clf.fit(X,Y)

# Check
strlog = "OvO produces %d classifiers" %(len(OvO_clf.estimators_))
print(strlog)
strlog = "OvR produces %d classifiers" %(len(OvR_clf.estimators_))
print(strlog)


OvO produces 21 classifiers
OvR produces 7 classifiers


## Validation

In [22]:
# Feature engineering, selection and rescaling to [0,1]
X_pred = scaler.transform(X_valid)
X_pred = poly.fit_transform(X_pred)
# PCA
#X_pred = pca.transform(X_pred)
Y_true = Y_valid.values.ravel()

# predict
Y_pred = clf.predict_proba(X_pred)

# predict
Y_pred_OvO = OvO_clf.predict(X_pred)
Y_pred_OvR = OvR_clf.predict(X_pred)


## Confusion Matrix

To check the performance of our model, we use Confusion Matrix as metric. This give us a general idea of how good is the model and let us compare it with the multiple combinations of feature that we have available.

In [23]:
# Performance metrics

from sklearn.metrics import confusion_matrix, precision_recall_fscore_support
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import roc_auc_score

conf_mat_OvO = confusion_matrix(Y_true,Y_pred_OvO)
hits_OvO = np.trace(conf_mat_OvO)
conf_mat_OvR = confusion_matrix(Y_true,Y_pred_OvR)
hits_OvR = np.trace(conf_mat_OvR)

# Print out
print("\nOvO confusion matrix:\n")
print(conf_mat_OvO)
print("\n")
print( "OvO Hits  = %d"%(hits_OvO) ) 
print( "OvO Fails = %d"%(Y_true.shape[0]-hits_OvO) )
print( "OvO Accuracy = %f"%(hits_OvO/(Y_true.shape[0])))
print( "OvO Cohen Accuracy = %f"%(cohen_kappa_score(Y_true, Y_pred_OvO)))
#print( "OvO AuC = ", (roc_auc_score(Y_true, Y_pred, multi_class="ovo")))

print("\nOvR confusion matrix:\n")
print(conf_mat_OvR)
print( "\n")
print( "OvR Hits  = %d"%(hits_OvR) ) 
print( "OvR Fails = %d"%(Y_true.shape[0]-hits_OvR) )
print( "OvR Accuracy = %f"%(hits_OvR/Y_true.shape[0]))
print( "OvR Cohen Accuracy = %f"%(cohen_kappa_score(Y_true, Y_pred_OvR)))
#print( "OvR AuC = ", (roc_auc_score(Y_true, Y_pred, multi_class="ovr")))





OvO confusion matrix:

[[6862 1327    2    0   81   13  189]
 [1076 9621   68    0  395  139   33]
 [   0   37 1149   42    9  193    0]
 [   0    0    8   92    0   10    0]
 [   5   65    5    0  301    4    0]
 [   0   30   66    8    6  585    0]
 [  95    6    0    0    0    0  719]]


OvO Hits  = 19329
OvO Fails = 3912
OvO Accuracy = 0.831677
OvO Cohen Accuracy = 0.734778

OvR confusion matrix:

[[7030 1225    7    0   40    9  163]
 [1052 9854   74    1  221  102   28]
 [  12   60 1172   31    5  150    0]
 [   1    1    8   94    0    6    0]
 [  10   82    6    0  277    4    1]
 [   8   42   90    9    9  537    0]
 [  82   16    0    0    1    0  721]]


OvR Hits  = 19685
OvR Fails = 3556
OvR Accuracy = 0.846995
OvR Cohen Accuracy = 0.756496
